In [238]:
import json
from pprint import pprint
from collections import namedtuple
data = json.load(open('./extractedFactsWithYear.json'), object_hook=lambda d: namedtuple('X', d.keys())(*d.values()))

In [239]:
sentences = [ article.sentence for article in data.articles]


In [3]:
import spacy
nlp = spacy.load('en_core_web_md')

In [317]:
entity_with_year_re = re.compile('''([A-Z][a-zA-Z'--(0-9)]*(?:\s[A-Z-0-9][a-zA-Z'--(0-9)]*)*((:|\sdel|\sfor|\sin|\sthe|\sof|\sat|\sand|\sis|\sor|\s&|\.|\sto)*(?:\s[A-Z-0-9][a-zA-Z'--(0-9)]*)+)*) \(([1-3][0-9]{3})\)''')
entity_re = re.compile('''([A-Z][a-zA-Z'--(0-9)]*(?:\s[A-Z-0-9][a-zA-Z-(0-9)]*)*((:|\sdel|\sfor|\sin|\sthe|\sof|\sat|\sto)*(?:\s[A-Z-0-9][a-zA-Z'--(0-9)]*)+)*)''')

In [334]:
def director_from_article_name(article_name):
    return next((ent.text for ent in nlp(article_name).ents if ent.label_ == 'PERSON'), '')
def parse_director_name(article_name, doc):
    director = next((ent.text for ent in doc.ents if ent.label_ == 'PERSON'), '')
    director_from_name = director = director_from_article_name(article_name)
    if director == '': return director_from_name
    if len(director.split(' ')) < 2 : return director_from_name
    return director
    
    
def firstRule(article_name,sentence):
    results = []
    doc = nlp(sentence)
    director = parse_director_name(article_name, doc)
    for s in entity_with_year_re.finditer(sentence):
        fact = {
                'director':director,
                'film' : s.group()[:-7],
                'year' : s.group()[-5:-1]
        }
        results.append(fact)
    return results

def extract(article):
    if re.search("\(([1-3][0-9]{3})\)",article.sentence):
        return firstRule(article.name, article.sentence)
    entities = [s.group() for s in entity_re.finditer(article.sentence)]
    years = [s.group() for s in re.finditer("([1-3][0-9]{3})",article.sentence)]
    if len(entities) == 1 and len(years) ==1:
        return [{
                'director':director_from_article_name(article.name),
                'film' : entities[0],
                'year' : years[0]
        }]
    if "directed" in article.sentence:
        return []
    if "directed" in article.sentence:
        return []
    else :
        return []

In [314]:
entities = [s.group() for s in entity_re.finditer(article.sentence)]
    years = [s.group() for s in re.finditer("([1-3][0-9]{3})",article.sentence)]
    if len(entities) == 1 and len(years) ==1:
        return [{
                'director':parse_director_name(article.name,''),
                'film' : entities[0],
                'year' : years[0]
        }]
    if "directed" in article.sentence:
        return []



sentence = "In the spring of 2009, Reitman directed Up in the Air starring George Clooney."
enities = [s.group() for s in entity_re.finditer(sentence)]
doc = nlp(sentence)
for word in doc:
    print("Word:", word.text)
    print("Tag:", word.tag_)
    print("Head:", word.head.text)
    print("Dependency relation:", word.dep_)
    print("Children:", list(word.children))
    print("")

Word: In
Tag: IN
Head: In
Dependency relation: ROOT
Children: [spring]

Word: the
Tag: DT
Head: spring
Dependency relation: det
Children: []

Word: spring
Tag: NN
Head: In
Dependency relation: pobj
Children: [the, of]

Word: of
Tag: IN
Head: spring
Dependency relation: prep
Children: [Reitman]

Word: 2009,
Tag: CD
Head: Reitman
Dependency relation: compound
Children: []

Word: Reitman
Tag: NNP
Head: of
Dependency relation: pobj
Children: [2009,, directed, starring]

Word: directed
Tag: VBD
Head: Reitman
Dependency relation: acl
Children: [Up, in]

Word: Up
Tag: RP
Head: directed
Dependency relation: prt
Children: []

Word: in
Tag: IN
Head: directed
Dependency relation: prep
Children: [Air]

Word: the
Tag: DT
Head: Air
Dependency relation: det
Children: []

Word: Air
Tag: NN
Head: in
Dependency relation: pobj
Children: [the]

Word: starring
Tag: VBG
Head: Reitman
Dependency relation: acl
Children: [Clooney.]

Word: George
Tag: NNP
Head: Clooney.
Dependency relation: compound
Children: [

In [305]:
tes ="In 2006, Greengrass directed United 93, a film based on the 11 September 2001 hijacking of United Airlines Flight 93."
test_re = re.compile('''([A-Z][a-zA-Z'--(0-9)]*(?:\s[A-Z-0-9][a-zA-Z-(0-9)]*)*((:|\sdel|\sfor|\sin|\sthe|\sof|\sat|\sto)*(?:\s[A-Z-0-9][a-zA-Z'--(0-9)]*)+)*)''')
for s in test_re.finditer(tes):
    print(s.group())
    

In 2006
Greengrass
United 93
September 2001
United Airlines Flight 93


In [242]:
def test(fact_dic, fact):    
    if fact['film'] not in fact_dic:
        print(fact_dic)
        print(fact)
        print(0)
        print()
        return 0
    if fact_dic[fact['film']]['director'] == fact['director']: 
        if fact_dic[fact['film']]['year'] == fact['year']:
            return 1
        else:
            print(fact_dic)
            print(fact)
            print(0.75)
            print()
            return 0.75
    print(fact_dic)
    print(fact)
    print(0.5)
    print()
    return 0.5

In [335]:
facts_amount = 0
final_score = 0
for article in data.articles:
    facts_amount = facts_amount + len(article.facts)
    extracted = extract(article)
    facts_dic = {f.film:{'director':f.director, 'year':f.year} for f in article.facts}
    for f in extracted:
        final_score = final_score + test(facts_dic, f)
final_score/facts_amount * 100

{'House of Games': {'director': 'David Mamet', 'year': '1987'}, 'Things Change': {'director': 'David Mamet', 'year': '1988'}, 'Homicide': {'director': 'David Mamet', 'year': '1991'}, 'Oleanna': {'director': 'David Mamet', 'year': '1994'}, 'The Spanish Prisoner': {'director': 'David Mamet', 'year': '1997'}, 'The Winslow Boy': {'director': 'David Mamet', 'year': '1999'}, 'State and Main': {'director': 'David Mamet', 'year': '2000'}, 'Heist': {'director': 'David Mamet', 'year': '2001'}, 'Spartan': {'director': 'David Mamet', 'year': '2004'}, 'Redbelt': {'director': 'David Mamet', 'year': '2008'}, 'Phil Spector': {'director': 'David Mamet', 'year': '2012'}}
{'director': 'David Mamet', 'film': 'London Critics Circle Film Awards), Things Change', 'year': '1988'}
0

{'House of Games': {'director': 'David Mamet', 'year': '1987'}, 'Things Change': {'director': 'David Mamet', 'year': '1988'}, 'Homicide': {'director': 'David Mamet', 'year': '1991'}, 'Oleanna': {'director': 'David Mamet', 'year': 

53.46534653465347

In [83]:
doc = nlp(sentences[0])
for ent in doc.ents:
    print(ent.text)

Paul Greengrass
three
Bourne
The Bourne Supremacy
The Bourne Ultimatum
Jason Bourne (2016);  
United 93
(2006),
the BAFTA Award for Best Director,
Academy Award
nomination, Green Zone
Phillips (2013).


In [81]:
46.03960396039604
47.524752475247524
48.51485148514851
49.504950495049506
50.0
52.475247524752476
53.46534653465347

Other
films
Paul Greengrass
has
directed
include
three
in
the
Bourne
action/thriller
series:
The Bourne Supremacy
(2004),
The Bourne Ultimatum
(2007),
and
Jason Bourne (2016);  
United 93
(2006),
for
which
he
won
the BAFTA Award for Best Director,
and
received
an
Academy Award
for
Best
Director
nomination, Green Zone
(2010)
and
Captain
Phillips (2013).


In [13]:
for word in doc:
    print("Word:", word.text)
    print("Tag:", word.tag_)
    print("Head:", word.head.text)
    print("Dependency relation:", word.dep_)
    print("Children:", list(word.children))
    print("")

Word: Reitman
Tag: NNP
Head: film
Dependency relation: poss
Children: ['s]

Word: 's
Tag: POS
Head: Reitman
Dependency relation: case
Children: []

Word: first
Tag: JJ
Head: film
Dependency relation: amod
Children: []

Word: feature
Tag: NN
Head: film
Dependency relation: compound
Children: []

Word: film
Tag: NN
Head: Thank
Dependency relation: nsubj
Children: [Reitman, first, feature, ,]

Word: ,
Tag: ,
Head: film
Dependency relation: punct
Children: []

Word: Thank
Tag: VBP
Head: Thank
Dependency relation: ROOT
Children: [film, You, for, ,, opened, .]

Word: You
Tag: PRP
Head: Thank
Dependency relation: dobj
Children: []

Word: for
Tag: IN
Head: Thank
Dependency relation: prep
Children: [Smoking]

Word: Smoking
Tag: NNP
Head: for
Dependency relation: pobj
Children: []

Word: ,
Tag: ,
Head: Thank
Dependency relation: punct
Children: []

Word: opened
Tag: VBN
Head: Thank
Dependency relation: conj
Children: [in]

Word: in
Tag: IN
Head: opened
Dependency relation: prep
Children: [2005]


In [94]:
results = []
for word in doc:
    if word.text == 'film':
        film = {}
        for child in word.children:
            print(child.ent_type_)
            if (child.ent_type_ == 'DATE' and child.dep_ == 'nummod'):
                film['date'] = child.text
            if (child.ent_type_ == 'PERSON' and child.dep_ == 'poss'):
                film['director'] = child.text[0:child.text.rindex("'")]
            if (child.ent_type_ == 'WORK_OF_ART' and child.dep_ == 'appos'):
                film['name'] = child.text.strip()
        print(film)
            

PERSON
DATE
WORK_OF_ART
{'director': 'Paul Greengrass', 'date': '1998', 'name': 'The Theory of Flight'}


In [103]:
for ent in doc.ents:
    print("Word:", list(ent.subtree)) 
    print("Word:", ent.text)
    print("Lemma:", ent.lemma_)
    print("Label:", ent.label_)

Word: [the spring of 2009]
Word: the spring of 2009
Lemma: the
Label: DATE
Word: [Reitman]
Word: Reitman
Lemma: reitman
Label: PERSON
Word: [the, Air]
Word: Air
Lemma: air
Label: ORG
Word: [George Clooney]
Word: George Clooney
Lemma: george
Label: PERSON


In [79]:
fo ent indir(doc)

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_py_tokens',
 '_realloc',
 '_vector',
 '_vector_norm',
 'cats',
 'char_span',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_disk',
 'get_extension',
 'get_lca_matrix',
 'has_extension',
 'has_vector',
 'is_parsed',
 'is_tagged',
 'mem',
 'merge',
 'noun_chunks',
 'noun_chunks_iterator',
 'print_tree',
 'sentiment',
 'sents',
 'set_extension',
 'similarity',
 'tensor',
 'text',
 'text_with_ws',
 'to_array',
 'to_bytes',
 'to_disk',
 'user_data',
 'user_hooks',
 'user_span_hooks',
 'user_token_hooks',
 'vector',
 've